# For SVM training

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import f1_score, classification_report
from sklearn.inspection import permutation_importance
from skopt import BayesSearchCV
import warnings

warnings.filterwarnings('ignore')

# ===================== Load Dataset =====================
df = pd.read_csv("Name of dataset.csv").drop_duplicates()
X = df.drop(columns=['Phase', 'composition'], errors='ignore')
y = df['Phase']

# ===================== Train-Test Split =====================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

# ===================== Bayesian Optimization =====================
pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC(class_weight='balanced', random_state=42))])
param_space = {
    'svc__C': (1e-3, 1e3, 'log-uniform'),
    'svc__gamma': (1e-4, 1e1, 'log-uniform'),
    'svc__kernel': ['rbf']
}
cv_bayes = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
bayes = BayesSearchCV(pipe, param_space, n_iter=30, cv=cv_bayes, scoring='f1_weighted', n_jobs=-1, random_state=42)
print("🔄 Running Bayesian Optimization...")
bayes.fit(X_train, y_train)

best_params = bayes.best_params_
print("✅ Best Hyperparameters:", best_params)

# ===================== Fix SVM with Best Hyperparameters =====================
best_pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC(
    C=best_params['svc__C'],
    gamma=best_params['svc__gamma'],
    kernel='rbf',
    class_weight='balanced',
    random_state=42
))])

# ===================== CV-Aware Permutation Importance (10 folds × 100 repeats) =====================
cv_perm = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
importances_cv = []

for fold, (train_idx, val_idx) in enumerate(cv_perm.split(X_train, y_train), start=1):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    best_pipe.fit(X_tr, y_tr)
    perm = permutation_importance(best_pipe, X_val, y_val, n_repeats=100, scoring='f1_weighted', random_state=42, n_jobs=-1)
    importances_cv.append(perm.importances_mean)

importances_cv = np.array(importances_cv)
mean_importance = importances_cv.mean(axis=0)
top_features = X_train.columns[np.argsort(mean_importance)[::-1]][:40]

X_train = X_train[top_features]
X_test = X_test[top_features]

# ===================== Correlation Pruning =====================
corr_matrix = X_train.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [col for col in upper_tri.columns if any(upper_tri[col] > 0.95)]
X_train = X_train.drop(columns=to_drop)
X_test = X_test.drop(columns=to_drop)

# ===================== True Greedy Forward Selection with CV =====================
selected_features, remaining_features = [], list(X_train.columns)
cv_f1_history, test_f1_history = [], []
best_cv_f1, no_improve_count, patience, min_delta = -np.inf, 0, 15, 0.001
cv_fs = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

while remaining_features:
    best_feature, best_step_f1 = None, -np.inf
    for feat in remaining_features:
        trial_features = selected_features + [feat]
        mean_cv_f1 = cross_val_score(best_pipe, X_train[trial_features], y_train, cv=cv_fs, scoring='f1_weighted', n_jobs=-1).mean()
        if mean_cv_f1 > best_step_f1:
            best_step_f1, best_feature = mean_cv_f1, feat

    selected_features.append(best_feature)
    remaining_features.remove(best_feature)
    cv_f1_history.append(best_step_f1)

    best_pipe.fit(X_train[selected_features], y_train)
    test_f1_history.append(f1_score(y_test, best_pipe.predict(X_test[selected_features]), average='weighted'))

    if best_step_f1 > best_cv_f1 + min_delta:
        best_cv_f1, no_improve_count = best_step_f1, 0
    else:
        no_improve_count += 1
    if no_improve_count >= patience:
        break

best_idx = np.argmax(cv_f1_history)
best_features = selected_features[:best_idx+1]

# ===================== Final Test Evaluation =====================
best_pipe.fit(X_train[best_features], y_train)
y_test_pred = best_pipe.predict(X_test[best_features])

best_cv_score = cv_f1_history[best_idx]

print("\n================ FINAL TEST PERFORMANCE =================")
print(f"Number of features : {len(best_features)}")
print(f"Selected features  : {best_features}")
print(f"Mean CV F1-weighted: {best_cv_score:.4f}")
print(f"Test F1-weighted   : {f1_score(y_test, y_test_pred, average='weighted'):.4f}")
print(classification_report(y_test, y_test_pred, target_names=['Non-single phase','Single phase']))


# For prediction

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

# ===================== Load Data =====================
df_train = pd.read_csv("Name of training dataset.csv").drop_duplicates()
df_pred  = pd.read_csv("Name of prediction dataset.csv")

# ===================== Features =====================
features = ['AR (A+B)', 'IE (A+B)', 'Delta (%)', 'AW(A+X)', 'AR(X/A)', 
            'AR(B/A)', 'AW(A+B)', 'AD(A-B)', 'AR(A-B)', 'AD(A-X)', 
            'VEC', 'IE (A-B)', 'AD(A+B)'] #Optimised feature set

X_train, y_train = df_train[features], df_train['Phase']
X_pred = df_pred[features]

# ===================== Train SVM =====================
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(C=1000, gamma=0.003726997693542054, kernel='rbf',
                class_weight='balanced', probability=True, random_state=42))
])
pipeline.fit(X_train, y_train)

# ===================== Predict Probabilities =====================
proba = pipeline.predict_proba(X_pred)[:, 1]
df_pred['Single_phase_proba'] = proba

# ===================== Apply Thresholds =====================
for t in [0.5, 0.6, 0.7]:
    df_pred[f'Pred_Phase_p{int(t*100)}'] = (proba >= t).astype(int)

# ===================== Save Results =====================
df_pred.to_csv("HESO_pred_cv10_Stratified.csv", index=False)
print("✅ Predictions saved with thresholds 0.5, 0.6, 0.7")


# CHGNet relaxation code after SQS generation

### The code assumes that the SQS generated structures are named as HEO1.vasp, HEO2.vasp....

In [ ]:
!pip install pandas pymatgen chgnet numpy tqdm

from google.colab import drive
drive.mount('/content/drive')

import os
import time
import pandas as pd
from pymatgen.core import Structure
from chgnet.model import StructOptimizer, CHGNet

# ------------------------
# CONFIGURATION
# ------------------------
start_idx = 1     # Start HEO index (inclusive)
end_idx = 5       # End HEO index (inclusive)

base_path = "/content/drive/MyDrive/HEO_predicted"
relax_dir = "relaxed_structures"
os.makedirs(relax_dir, exist_ok=True)

# ------------------------
# Load CHGNet Model
# ------------------------
print("🔧 Initializing CHGNet model...")
chgnet_model = CHGNet.load()
chgnet_relaxer = StructOptimizer()

# ------------------------
# Relax Structures
# ------------------------
csv_path = "relaxation_results.csv"
all_entries = []

print(f"\n🔄 Starting CHGNet relaxation for HEO{start_idx} to HEO{end_idx}...\n")

for i in range(start_idx, end_idx + 1):
    heo_file = f"HEO{i}.vasp"
    vasp_path = os.path.join(base_path, heo_file)

    if not os.path.exists(vasp_path):
        print(f"⚠️  Skipping missing file: {heo_file}")
        continue

    try:
        struct = Structure.from_file(vasp_path)
        print(f"🔧 Relaxing {heo_file}...")

        start_time = time.time()
        result = chgnet_relaxer.relax(struct, fmax=0.005, relax_cell=True, verbose=True)
        end_time = time.time()

        relaxed_struct = result["final_structure"]
        final_energy = float(result["trajectory"].energies[-1])
        final_fmax = float(result["trajectory"].forces[-1].max())
        relax_time_sec = end_time - start_time

        # Extract lattice information
        lattice = relaxed_struct.lattice
        a, b, c = lattice.a, lattice.b, lattice.c
        alpha, beta, gamma = lattice.alpha, lattice.beta, lattice.gamma
        volume = lattice.volume

        save_name = heo_file.replace(".vasp", "_rlx.vasp")
        save_path = os.path.join(relax_dir, save_name)
        relaxed_struct.to(fmt="poscar", filename=save_path)

        row = {
            "HEO_file": heo_file,
            "relaxed_file": save_name,
            "relaxed_energy (eV)": final_energy,
            "a (Å)": a,
            "b (Å)": b,
            "c (Å)": c,
            "alpha (°)": alpha,
            "beta (°)": beta,
            "gamma (°)": gamma,
            "volume (Å³)": volume,
            "time_taken (s)": relax_time_sec,
            "final_fmax (eV/Å)": final_fmax
        }
        all_entries.append(row)
        print(f"  ✅ Relaxed {heo_file} | Energy = {final_energy:.4f} eV | Time = {relax_time_sec:.1f} s\n")

    except Exception as e:
        print(f"  ❌ Relaxation failed for {heo_file}: {e}")

# ------------------------
# Append to CSV
# ------------------------
if all_entries:
    df = pd.DataFrame(all_entries)
    if os.path.exists(csv_path):
        df.to_csv(csv_path, mode='a', index=False, header=False)
    else:
        df.to_csv(csv_path, index=False)
    print(f"💾 Appended results to: {csv_path}")
else:
    print("❗ No successful relaxations to record.")

print("\n🎉 Task complete!")


from zipfile import ZipFile
from google.colab import files

# === ZIP the relaxed structures folder ===
zip_filename = "relaxed_structures.zip"
with ZipFile(zip_filename, 'w') as zipf:
    for root, _, files_list in os.walk(relax_dir):
        for file in files_list:
            filepath = os.path.join(root, file)
            arcname = os.path.relpath(filepath, relax_dir)
            zipf.write(filepath, arcname=os.path.join("relaxed_structures", arcname))

# === Download the zip and the CSV ===
print("\n📦 Downloading results...")
files.download(zip_filename)         # relaxed structures
files.download(csv_path)             # relaxation_results.csv

# SynthNN score prediction

In [ ]:
!git clone https://github.com/antoniuk1/SynthNN.git
%cd SynthNN

#from SynthNN.utils import synthNN_predict
import numpy as np
from SynthNN.utils import get_features, synthNN_predict
import tensorflow as tf
import os

#Option 2: Read in formulas from text file
data=np.loadtxt('input_formulas.txt',dtype=str) #Paste the composition like Mg0.2Mn0.2Cu0.2Ni0.2Zn0.2Al2O4
output_file_name='output_formula_preds.txt' 
saved_model_dir='./Trained_models/Paper_final_model/'

synthNN_predict(data,output_file_name, saved_model_dir)

# Code for XRD generation

### This code assumes you have a folder named 'relaxed_structures_fil with CHGNet relaxed structures and material ID (HEOx_rlx.vasp) x=1,2,3... Rest other csv's are provided.

In [ ]:
# === Imports ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pymatgen.core import Structure
from pymatgen.analysis.diffraction.xrd import XRDCalculator
from pymatgen.ext.matproj import MPRester
import os

# === Your MP API Key (paste it here) ===
MAPI_KEY = "Your API Key"

# === Initialize objects ===
mpr = MPRester(MAPI_KEY)
xrd_calc = XRDCalculator(wavelength="CuKa")  # default λ = 1.5406 Å

# === Peak filtering function ===
def filter_peaks(pattern, threshold=0.04):
    max_intensity = max(pattern.y)
    return [(round(t, 2), round(i / max_intensity, 3)) 
            for t, i in zip(pattern.x, pattern.y) if i / max_intensity >= threshold]


In [ ]:
def compute_similarity(ref_peaks, test_peaks, theta_tol=0.6, intensity_diff_weight=0.1):
    """
    Compare two XRD patterns using only peak positions (2θ) and intensities.
    Penalizes:
    - missing reference peaks (0.05 per peak)
    - extra peaks in test with intensity > 0.1
    - intensity differences between matched peaks (scaled by intensity_diff_weight)

    Parameters:
        ref_peaks (list of (theta, intensity)) - Reference pattern (e.g., from Materials Project)
        test_peaks (list of (theta, intensity)) - Test pattern (e.g., from CHGNet-relaxed structure)
        theta_tol (float) - Matching tolerance for 2θ in degrees
        intensity_diff_weight (float) - weight factor for intensity difference penalty

    Returns:
        similarity_score (float between 0 and 1)
    """
    penalty = 0.0

    # === Match each reference peak and accumulate intensity difference penalty ===
    matched_test_indices = set()
    for i_ref, (theta_ref, inten_ref) in enumerate(ref_peaks):
        matched = False
        for i_test, (theta_test, inten_test) in enumerate(test_peaks):
            if abs(theta_test - theta_ref) <= theta_tol:
                matched = True
                matched_test_indices.add(i_test)
                # Intensity difference penalty (normalized intensities between 0-1)
                inten_diff = abs(inten_ref - inten_test)
                penalty += inten_diff * intensity_diff_weight
                break
        if not matched:
            penalty += 0.05  # Penalize for each unmatched reference peak

    # === Penalize extra peaks in test with intensity > 0.1 ===
    for i_test, (theta_test, inten_test) in enumerate(test_peaks):
        if i_test not in matched_test_indices and inten_test > 0.1:
            penalty += 0.005

    # === Final similarity score ===
    score = 1 / (1 + penalty)
    return score


In [ ]:
# === Load input data ===
relaxed_df = pd.read_csv("Final selected structures.csv") #CSV with final selected structures composition (A1-A5 & B-site), SVM score, SynthNN score
mp_df = pd.read_csv("normal_spinels.csv") 

# Filter out rows where Synthpred < 0.6
#relaxed_df = relaxed_df[relaxed_df['SynthPred'] >= 0.6]

# === For trial, limit to first 5 ===
start_idx = 0
end_idx = 38
trial_df = relaxed_df.iloc[start_idx:end_idx]

results = []
patterns_to_plot = []

for _, row in trial_df.iterrows():
    relaxed_path = os.path.join("relaxed_structures_fil", row["relaxed_file"])
    B_site = row["B"]
    
    # Load relaxed structure
    try:
        struct_rlx = Structure.from_file(relaxed_path)
        xrd_rlx = xrd_calc.get_pattern(struct_rlx)
        peaks_rlx = filter_peaks(xrd_rlx)
    except:
        print(f"Error loading: {relaxed_path}")
        continue

    # Find MP reference spinels with same B
    matches = mp_df[mp_df["B"] == B_site]
    best_score = -1
    best_mp_id = None
    best_formula = None
    best_ref_peaks = []

    for _, mp_row in matches.iterrows():
        mp_id = mp_row["material_id"]
        try:
            struct_mp = mpr.get_structure_by_material_id(mp_id)
            xrd_mp = xrd_calc.get_pattern(struct_mp)
            peaks_mp = filter_peaks(xrd_mp)
            score = compute_similarity(peaks_mp, peaks_rlx)
            if score > best_score:
                best_score = score
                best_mp_id = mp_id
                best_formula = mp_row["formula"]
                best_ref_peaks = peaks_mp
        except:
            continue
    
    results.append({
        "relaxed_file": row["relaxed_file"],
        "best_mp_id": best_mp_id,
        "best_formula": best_formula,
        "similarity_score": best_score
    })
    
    patterns_to_plot.append((row["relaxed_file"], peaks_rlx, best_mp_id, best_ref_peaks))


In [ ]:
import matplotlib.pyplot as plt
import re

def sanitize_filename(s):
    # Replace any character not alphanumeric or _ with _
    return re.sub(r'[^a-zA-Z0-9_-]', '_', s.replace(' ', '_'))

def to_subscript(s):
    sub_map = str.maketrans("0123456789.", "₀₁₂₃₄₅₆₇₈₉⋅")
    return s.translate(sub_map)

plots_per_fig = 25
n_plots = len(patterns_to_plot)
n_figs = (n_plots - 1) // plots_per_fig + 1

for fig_idx in range(n_figs):
    start_idx = fig_idx * plots_per_fig
    end_idx = min(start_idx + plots_per_fig, n_plots)
    current_patterns = patterns_to_plot[start_idx:end_idx]
    
    fig, axes = plt.subplots(5, 5, figsize=(25, 25), dpi=500)
    axes = axes.flatten()
    
    for i, (relaxed_file, peaks_rlx, mp_id, peaks_mp) in enumerate(current_patterns):
        ax = axes[i]

        # XRD peak positions and normalized intensities
        x_rlx, y_rlx = zip(*peaks_rlx) if peaks_rlx else ([], [])
        x_mp, y_mp = zip(*peaks_mp) if peaks_mp else ([], [])

        # Plot relaxed (HEO) structure — solid blue sticks
        for x, y in zip(x_rlx, y_rlx):
            ax.vlines(x, 0, y, color='blue', linewidth=1.5)

        # Plot MP reference structure — dashed red sticks
        for x, y in zip(x_mp, y_mp):
            ax.vlines(x, 0, y, color='red', linestyle='dashed', linewidth=1.5)

        # Add baseline at y=0
        ax.axhline(0, color='black', linewidth=0.8)

        # Get composition string from relaxed_df
        base_name = relaxed_file.replace('_rlx.vasp', '')
        composition = relaxed_df.loc[relaxed_df['relaxed_file'] == relaxed_file, 'composition'].values[0]
        composition_sub = to_subscript(composition)

        # Get MP formula and subscript
        mp_row = next((r for r in results if r["relaxed_file"] == relaxed_file), None)
        mp_formula = mp_row["best_formula"] if mp_row else "?"
        mp_formula_sub = to_subscript(mp_formula)
        mp_id_display = mp_row["best_mp_id"] if mp_row else "?"

        # Similarity score for title
        score = mp_row["similarity_score"]
        score_str = f"{score:.2f}" if score is not None else "N/A"

        # Title and labels
        title_str = f"{base_name} vs {mp_formula} ({score_str})"
        ax.set_title(title_str, fontsize=10)
        ax.set_xlabel("2θ (°)", fontsize=9)
        ax.set_ylabel("Normalized Intensity", fontsize=9)
        ax.set_xlim(20, 60)

        # Legend with subscripts
        legend_labels = [
            f"{composition_sub}",
            f"{mp_formula_sub} ({mp_id_display})"
        ]
        ax.legend(legend_labels, fontsize=8)

        # Save individual subplot as separate figure
        fig_single, ax_single = plt.subplots(figsize=(8, 5), dpi=500)

        # Plot same data in individual subplot figure
        for x, y in zip(x_rlx, y_rlx):
            ax_single.vlines(x, 0, y, color='blue', linewidth=1.5)
        for x, y in zip(x_mp, y_mp):
            ax_single.vlines(x, 0, y, color='red', linestyle='dashed', linewidth=1.5)
        ax_single.axhline(0, color='black', linewidth=0.8)

        ax_single.set_title(title_str, fontsize=12)
        ax_single.set_xlabel("2θ (°)", fontsize=10)
        ax_single.set_ylabel("Normalized Intensity", fontsize=10)
        ax_single.set_xlim(20, 60)
        ax_single.legend(legend_labels, fontsize=10)
        plt.tight_layout()

        # Sanitize filename and save
        filename = sanitize_filename(title_str) + ".tiff"
        fig_single.savefig(filename, dpi=500)
        plt.close(fig_single)

    # Hide any unused subplots in the grid
    for j in range(i + 1, plots_per_fig):
        axes[j].axis('off')

    plt.tight_layout()
    combined_filename = f"XRD_similarity_combined_part_{fig_idx+1}.tiff"
    fig.savefig(combined_filename, dpi=500)
    plt.show()
    plt.close(fig)
